## Add dependencies

In [1]:
%pip install -qU sqlalchemy pydantic google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.4/206.4 kB 10.7 MB/s eta 0:00:00


In [2]:
%load_ext sql
%config SqlMagic.style = '_DEPRECATED_DEFAULT'

## Set up database

In [3]:
from sqlalchemy import Column, Integer, String, ForeignKey
from sqlalchemy.orm import declarative_base, relationship

Base = declarative_base()


class MemberDB(Base):
    __tablename__ = "members"

    id = Column(Integer, primary_key=True, index=True)
    name = Column(String, index=True)
    email = Column(String)
    age = Column(Integer)
    phone = Column(String)
    full_address = Column(String)

    interests = relationship("MemberInterestDB", back_populates="member")


class MemberInterestDB(Base):
  __tablename__ = "member_interests"

  id = Column(Integer, primary_key=True, index=True)
  member_id = Column(Integer, ForeignKey("members.id"), index=True)
  interest = Column(String)

  member = relationship("MemberDB", back_populates="interests")


In [4]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

# Local sqlite database for demo purposes.
DATABASE_URL = "sqlite:///./test.db"

engine = create_engine(DATABASE_URL)

# Create all tables
Base.metadata.create_all(bind=engine)

SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

Load the IPython sqlite tool and peek into the DB.

In [5]:
%sql sqlite:///./test.db

In [6]:
%%sql
SELECT name FROM sqlite_master WHERE type='table';

 * sqlite:///./test.db
Done.


name
members
member_interests


In [7]:
%%sql
SELECT * FROM members;

 * sqlite:///./test.db
Done.


id,name,email,age,phone,full_address


You should see no rows, but the query should execute and display the table columns.

## Set up logical model

The logical model represents the application's data structure, optimized for its workflows. This often differs from the physical database schema, as applications rarely need all database fields for every task. For instance, a `password` column, while essential in the database, wouldn't be included when displaying a user record in a UI.

In [8]:
import enum

from pydantic import BaseModel, ConfigDict

class InterestChoice(enum.Enum):
  ARTS = "arts_and_crafts"
  BUSHCRAFT = "bushcraft"
  COOKING = "cooking"
  KNOTS = "knot_tying"
  MUSIC = "music"
  NATURE = "nature_and_plants"
  PHOTOGRAPHY = "photograph"
  SPORTS = "sports"
  TECHNOLOGY = "tech"


class Interest(BaseModel):
    model_config = ConfigDict(from_attributes=True)

    interest: InterestChoice


class Member(BaseModel):
    model_config = ConfigDict(from_attributes=True)

    id: int | None = None
    name: str
    email: str
    age: int
    phone: str
    full_address: str
    interests: list[Interest]


Define a helper to insert a logical instance into the database.

In [10]:
def create_member_with_interests(db: SessionLocal, member_data: Member):
    """Insert the provided member, including any associated interests in a DB transaction."""
    db_member = MemberDB(
        name=member_data.name,
        email=member_data.email,
        age=member_data.age,
        phone=member_data.phone,
        full_address=member_data.full_address
    )

    for interest in member_data.interests:
        db_interest = MemberInterestDB(interest=interest.interest.value, member=db_member)
        db.add(db_interest)

    db.add(db_member)
    db.commit()
    db.refresh(db_member)
    return db_member

Test that the insert helper works.

In [11]:
db = SessionLocal()

new_member = Member(
    name="John Doe",
    email="john.doe@example.com",
    age=29,
    phone="123-456-7890",
    full_address="123 Main St",
    interests=[Interest(interest=InterestChoice.COOKING), Interest(interest=InterestChoice.BUSHCRAFT)]
)

db_row = create_member_with_interests(db, new_member)

In [12]:
%%sql
select * from members where email = 'john.doe@example.com';

 * sqlite:///./test.db
Done.


id,name,email,age,phone,full_address
1,John Doe,john.doe@example.com,29,123-456-7890,123 Main St


In [13]:
%%sql
select * from member_interests where member_id = 1;

 * sqlite:///./test.db
Done.


id,member_id,interest
1,1,cooking
2,1,bushcraft


## Set up the Gemini API

Load and configure the Gemini SDK client.

In [17]:
import os
from google.colab import userdata

os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

In [18]:
from google import genai
from google.genai import types

client = genai.Client()

Download a sample image of a completed form. You can find the original form [here](https://docs.google.com/document/d/1u7VbeT9YTiqLsu952GFRz5ORn6HqQ4J1tNbA5PESpKg/edit?tab=t.0) if you want to try your own out.

In [14]:
!wget https://storage.googleapis.com/generativeai-downloads/images/new_member.jpg

--2025-06-24 06:42:08--  https://storage.googleapis.com/generativeai-downloads/images/new_member.jpg
Resolving storage.googleapis.com (storage.googleapis.com)... 192.178.163.207, 74.125.20.207, 108.177.98.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|192.178.163.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1429960 (1.4M) [image/jpeg]
Saving to: ‘new_member.jpg’

new_member.jpg      100%[===================>]   1.36M  --.-KB/s    in 0.008s  

2025-06-24 06:42:08 (172 MB/s) - ‘new_member.jpg’ saved [1429960/1429960]



In [15]:
import PIL.Image

form_photo = PIL.Image.open('new_member.jpg')

form_photo

Make the Gemini API call to process the image.

In [ ]:
response = client.models.generate_content(
    model='gemini-2.5-flash',
    config=types.GenerateContentConfig(
        response_mime_type='application/json',
        response_schema=Member,
        system_instruction="The current date is May 11, 2025"
    ),
    contents=form_photo,
)

new_member = response.parsed
new_member

In [23]:
print(response.text)

{"id":null,"name":"Wilson Spalding","email":"will@example.com","age":55,"phone":"0491570156","full_address":"1 Basketball Ct. Sportsville","interests":[{"interest":"music"},{"interest":"nature_and_plants"},{"interest":"sports"}]}


Now insert the returned object directly into the database.

In [24]:
create_member_with_interests(db, new_member);

Finally, test that it landed.

In [25]:
%%sql
select * from members where email = 'Will@example.com';

 * sqlite:///./test.db
Done.


id,name,email,age,phone,full_address
2,Wilson Spalding,will@example.com,55,0491570156,1 Basketball Ct. Sportsville


It worked!